In [8]:
from google.colab import drive
drive.mount('/content/mnt')

Drive already mounted at /content/mnt; to attempt to forcibly remount, call drive.mount("/content/mnt", force_remount=True).


In [9]:
pip install opacus

In [10]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset, Subset, random_split
from opacus import PrivacyEngine

import copy
import math

In [12]:
class FederatedClient:
    def __init__(self, client_id, model, train_data, dp_config=None, device='cpu'):
        self.client_id = client_id
        self.model = model.to(device)
        self.device = device
        self.train_data = train_data
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.SGD(self.model.parameters(), lr=0.05)
        self.dp_enabled = dp_config is not None

        if self.dp_enabled:
            # wrap optimizer with Opacus PrivacyEngine
            self.privacy_engine = PrivacyEngine()
            self.model, self.optimizer, self.train_data = self.privacy_engine.make_private(
                module=self.model,
                optimizer=self.optimizer,
                data_loader=self.train_data,
                noise_multiplier=dp_config.get("noise_multiplier", 1.2),
                max_grad_norm=dp_config.get("max_grad_norm", 1.0),
            )

    def local_train(self, epochs=1):
        self.model.train()

        for _ in range(epochs):
            for data, target in self.train_data:
                data, target = data.to(self.device), target.to(self.device)
                self.optimizer.zero_grad()
                output = self.model(data)
                loss = self.criterion(output, target)
                loss.backward()
                self.optimizer.step()

        # Calculate model updates
        return (self.model._module if self.dp_enabled else self.model).state_dict()

In [13]:
class FederatedServer:
    def __init__(self, global_model, device='cpu'):
        self.global_model = global_model
        self.clients = []
        self.round = 0
        self.device = device

    def add_client(self, client):
        self.clients.append(client)

    def federated_round(self, local_epochs=1):
        client_updates = []

        for client in self.clients:
            # sync global weights to clients
            client_model_state = self.global_model.state_dict()
            if client.dp_enabled:
                client.model._module.load_state_dict(client_model_state)
            else:
                client.model.load_state_dict(client_model_state)
            update = client.local_train(local_epochs)
            client_updates.append(update)

        # Aggregate with averaging
        aggregated_state = {}
        num_clients = len(client_updates)
        for name in client_updates[0]:
            stacked = torch.stack([u[name].to(self.device) for u in client_updates])
            aggregated_state[name] = torch.mean(stacked, dim=0).to(self.device)

        # Update global model
        self.global_model.load_state_dict(aggregated_state)
        self.round += 1

In [14]:
def load_mnist_csv(path):
    """
    Load MNIST-like CSV file (label in col 0, pixels in cols 1:785).
    """
    df = pd.read_csv(path, header=None).values
    y = torch.tensor(df[:, 0].astype(np.int64))
    X = torch.tensor(df[:, 1:].astype(np.float32))
    X = X / 255.0
    return X, y

def make_test_loader(path="/content/sample_data/mnist_test.csv", batch_size=256):
    X, y = load_mnist_csv(path)
    test_ds = TensorDataset(X, y)
    return DataLoader(test_ds, batch_size=batch_size, shuffle=False)

def make_federated_loaders(
    num_clients=3,
    batch_size=64,
    shuffle_seed=42,
    path="/content/sample_data/mnist_train_small.csv",
    test_path="/content/sample_data/mnist_test.csv"
):
    X, y = load_mnist_csv(path)

    # Create test loader from external test set
    test_loader = make_test_loader(test_path, batch_size=256)

    # Partition training data evenly across clients
    n = len(X)
    per_client = math.ceil(n / num_clients)
    order = np.random.RandomState(shuffle_seed).permutation(n)

    client_loaders = []
    dataset = TensorDataset(X, y)

    for c in range(num_clients):
        start, end = c * per_client, min((c + 1) * per_client, n)
        if start >= end:
            break
        subset_idx = order[start:end]
        subset = Subset(dataset, subset_idx.tolist())
        loader = DataLoader(subset, batch_size=batch_size, shuffle=True, drop_last=False)
        client_loaders.append(loader)

    return client_loaders, test_loader


In [19]:
def evaluate(model, test_loader, device=None):
    model.eval()
    device = device or next(model.parameters()).device
    correct, total = 0, 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = output.argmax(dim=1)
            correct += (pred == target).sum().item()
            total += target.size(0)
    return correct / total

In [16]:
class SimpleNN(nn.Module):
    def __init__(self, input_size=784, hidden_size=800, num_classes=10):
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [24]:
def main():
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Using device: {device}")

    dp_config = {
        "noise_multiplier": 1.2,
        "max_grad_norm": 1.0,
    }

    global_model = SimpleNN().to(device)
    server = FederatedServer(global_model, device=device)

    # split train/test
    client_data, test_loader = make_federated_loaders(num_clients=4,
                                                      batch_size=64,
                                                      shuffle_seed=42)

    for client_id, train_data in enumerate(client_data):
        # Each client gets a copy of the global model
        client_model = copy.deepcopy(global_model).to(device)
        # Set dp_config to None to train without Differential Privacy
        client = FederatedClient(client_id, client_model, train_data, dp_config=None, device=device)
        server.add_client(client)

    # Run FL rounds
    for r in range(10):
        print(f"\n--- Federated Round {r+1} ---")
        server.federated_round(local_epochs=2)
        acc = evaluate(server.global_model, test_loader, device=device)
        print(f"Test Accuracy after round {r+1}: {acc:.4f}")

    print("\n Federated learning with differential privacy complete.")

if __name__ == "__main__":
    main()

Using device: cuda

--- Federated Round 1 ---
Test Accuracy after round 1: 0.7354

--- Federated Round 2 ---
Test Accuracy after round 2: 0.8079

--- Federated Round 3 ---
Test Accuracy after round 3: 0.8492

--- Federated Round 4 ---
Test Accuracy after round 4: 0.8649

--- Federated Round 5 ---
Test Accuracy after round 5: 0.8766

--- Federated Round 6 ---
Test Accuracy after round 6: 0.8853

--- Federated Round 7 ---
Test Accuracy after round 7: 0.8901

--- Federated Round 8 ---
Test Accuracy after round 8: 0.8934

--- Federated Round 9 ---
Test Accuracy after round 9: 0.8961

--- Federated Round 10 ---
Test Accuracy after round 10: 0.8994

 Federated learning with differential privacy complete.
